# Anomalies collector - step by step tutorial

<a href="https://colab.research.google.com/github/netdata/netdata-community/blob/main/netdata-agent-api/netdata-pandas/anomalies_collector_tutorial.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook will walk through, step by step, a worked example of how the [netdata anomalies collector](https://github.com/andrewm4894/netdata/tree/anomalies-collector/collectors/python.d.plugin/anomalies) works under the hood. 

**Note**: you can click the "Open in Colab" button above to open this notebook in [Google Colab](https://colab.research.google.com/notebooks/intro.ipynb#recent=true) where you can just get going with it without having to set up python enviornments or any messy stuff like that.

In [166]:
# uncomment the below line to install required packages if needed.
#!pip install netdata-pandas==0.0.28 numba==0.50.1 scikit-learn==0.23.2 pyod==0.8.3

## Overview

There are three main functions in the anomalies collector that do most of the work:

- make_features(): a function to take raw data and create our feature vector used in the models.
- train(): a function to take our training data and train our models, one for each chart. 
- predict(): a function to take the most recent data and run it through the trained model's predict() method to get our anomsly probability and anomaly flag.  

In [167]:
import time
from datetime import datetime
import re

from IPython.display import display, Markdown
import numpy as np
import pandas as pd
from netdata_pandas.data import get_data, get_allmetrics
from pyod.models.hbos import HBOS
from pyod.models.pca import PCA


def make_features(df, lags_n, diffs_n, smooth_n):
    """Given a pandas dataframe preprocess it to take differences, add smoothing, and lags as specified. 
    """
    if diffs_n >= 1:
        df = df.diff(diffs_n).dropna()
    if smooth_n >= 2:
        df = df.rolling(smooth_n).mean().dropna()
    if lags_n >= 1:
        df_columns_new = [f'{col}_lag{n}' for n in range(lags_n+1) for col in df.columns]
        df = pd.concat([df.shift(n) for n in range(lags_n + 1)], axis=1).dropna()
        df.columns = df_columns_new
    # sort columns to have lagged values next to each other for clarity
    df = df.reindex(sorted(df.columns), axis=1)
    return df

In the next cell we will define all the inputs we will use in this tutorial. Feel free to play with them once you are familiar with how it all hangs together. 

In [168]:
# inputs
host = 'london.my-netdata.io'
charts_in_scope = ['system.cpu', 'system.load'] # for this tutorial we will just use two charts, and so two models
model = 'pca'
train_n_secs = 14400
contamination = 0.001
offset_n_secs = 0
lags_n = 5
smooth_n = 3
diffs_n = 1
n_prediction_steps = 20

In [169]:
# initialize a model for each chart
if model == 'pca':
    models = {c: PCA(contamination=contamination) for c in charts_in_scope}
else:
    models = {c: HBOS(contamination=contamination) for c in charts_in_scope}

## Get training data

The first thing we need to do is get our raw training data for each chart we want to build a mnodel for.

In [170]:
# define the window for the training data to pull
before = int(datetime.now().timestamp()) - offset_n_secs
after =  before - train_n_secs

# get the training data
df_train = get_data(hosts=host, charts=charts_in_scope, after=after, before=before, sort_cols=True, numeric_only=True)
print(df_train.shape)
df_train.head()

(14400, 12)


,system.cpu|guest,system.cpu|guest_nice,system.cpu|iowait,system.cpu|irq,system.cpu|nice,system.cpu|softirq,system.cpu|steal,system.cpu|system,system.cpu|user,system.load|load1,system.load|load15,system.load|load5
time_idx,,,,,,,,,,,,
1603824118,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.501253,0.501253,NaN,NaN,NaN
1603824119,0.0,0.0,0.0,0.0,0.0,0.00,0.00,1.005025,0.502513,NaN,NaN,NaN
1603824120,0.0,0.0,0.0,0.0,0.0,0.25,0.25,0.500000,0.500000,0.03,0.04,0.09
1603824121,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.755668,0.251889,0.03,0.04,0.09
1603824122,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.750000,1.250000,0.03,0.04,0.09


Above we can see our raw training data is just a pandas `DataFrame` with a timestamp index and a column for each metric from our `charts_in_scope` list. 

## Preprocess training data

Before we train our model we will first do some preprocessing to the raw data to create a "feature vector" to try and encode a more flexible and powerful representation for the model to work with as opposed to just looking at the most recently observed values in isolation. The idea here is to give the model some extra information so that it may spot more complex and interesting anomalies as opposed to just spikes where one metric is a very high or very low value.   

In [171]:
df_train_processed = make_features(df_train, lags_n, diffs_n, smooth_n)
print(df_train_processed.shape)
df_train_processed.head()

(14390, 72)


,system.cpu|guest_lag0,system.cpu|guest_lag1,system.cpu|guest_lag2,system.cpu|guest_lag3,system.cpu|guest_lag4,system.cpu|guest_lag5,system.cpu|guest_nice_lag0,system.cpu|guest_nice_lag1,system.cpu|guest_nice_lag2,system.cpu|guest_nice_lag3,...,system.load|load1_lag2,system.load|load1_lag3,system.load|load1_lag4,system.load|load1_lag5,system.load|load5_lag0,system.load|load5_lag1,system.load|load5_lag2,system.load|load5_lag3,system.load|load5_lag4,system.load|load5_lag5
time_idx,,,,,,,,,,,,,,,,,,,,,
1603824128,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1603824129,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1603824130,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1603824131,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1603824132,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [172]:
# Lets look at how the shape of our data has changed due to preprocessing
print(f'df_train shape is {df_train.shape}')
print(f'df_train_processed is {df_train_processed.shape}')
n_cols_added = len(df_train_processed.columns)-len(df_train.columns)
print(f'make_features has added {n_cols_added} new columns, one for each lags_n ({df_train.shape[1]}*{lags_n}={n_cols_added})')

df_train shape is (14400, 12)
df_train_processed is (14390, 72)
make_features has added 60 new columns, one for each lags_n (12*5=60)


So as you can see from the above output, our preprocessing has added a new column for each `lags_n` specified. And we have lost a few rows due to `smooth_n` and `diffs_n`

To be super clear lets look at the first few rows of training data for a specific metric before and after preprocessing. 

In [173]:
metric = 'system.cpu|user'
print('raw data')
display(df_train[df_train.columns[df_train.columns.str.startswith(metric)]].head(1 + lags_n + smooth_n + diffs_n))
print('processed data')
display(df_train_processed[df_train_processed.columns[df_train_processed.columns.str.startswith(metric)]].head(1))
print('manualy calculated')
display(df_train[df_train.columns[df_train.columns.str.startswith(metric)]].diff(diffs_n).dropna().rolling(smooth_n).mean().head(1 + lags_n + smooth_n + diffs_n).tail(1))

raw data


,system.cpu|user
time_idx,
1603824118,0.501253
1603824119,0.502513
1603824120,0.500000
1603824121,0.251889
1603824122,1.250000
1603824123,0.503778
1603824124,0.755668
1603824125,1.005025
1603824126,0.751880


processed data


,system.cpu|user_lag0,system.cpu|user_lag1,system.cpu|user_lag2,system.cpu|user_lag3,system.cpu|user_lag4,system.cpu|user_lag5
time_idx,,,,,,
1603824128,-0.085632,0.167926,0.0827,-0.081658,0.167926,0.001259


manualy calculated


,system.cpu|user
time_idx,
1603824128,-0.085632


Above you can see how one raw metric value is now being preprocessed to be a vector of `lags_n` differenced and smoothed values. It is this matrix of smoothed differences that the model will use for both training and during a predict step. 

## Train models

Now that we have our preprocessed training data we will train a model for each chart using this preprocessed feature represnetation that represents each time step for each chart as a lagged, smoothed metrix of differences for each chart.  

In [174]:
# loop over each chart in scope and train a model for each
for chart in charts_in_scope:
    X_train = df_train_processed[df_train_processed.columns[df_train_processed.columns.str.startswith(chart)]].values
    print(f'train model for {chart} using X_train of {X_train.shape}')
    models[chart] = models[chart].fit(X_train)

train model for system.cpu using X_train of (14390, 54)
train model for system.load using X_train of (14390, 18)


So we have now trained our models, one for each chart based on our preprocessed training data. To be concrete we will look at some example obvervations our model has been trained on. 

In [175]:
# lets look at the first matrix or "feature vector" for our first chart
obs_n = 0
print(f'timestamp={df_train_processed[df_train_processed.columns[df_train_processed.columns.str.startswith(charts_in_scope[0])]].index[obs_n]}')
print(f'feature vector for {obs_n}th training observation:')
print(df_train_processed[df_train_processed.columns[df_train_processed.columns.str.startswith(charts_in_scope[0])]].values[obs_n]) 

timestamp=1603824128
feature vector for 0th training observation:
[ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
 -0.00062657  0.          0.          0.0837521   0.         -0.08333333
  0.08312553  0.          0.          0.          0.         -0.08333333
 -0.0018797  -0.16792613 -0.0012626   0.00125627  0.08396307  0.0852225
 -0.0856318   0.1679261   0.08270047 -0.0816583   0.1679261   0.00125943]


In [176]:
# and the next one
obs_n = 1
print(f'timestamp={df_train_processed[df_train_processed.columns[df_train_processed.columns.str.startswith(charts_in_scope[0])]].index[obs_n]}')
print(f'feature vector for {obs_n}th training observation:')
print(df_train_processed[df_train_processed.columns[df_train_processed.columns.str.startswith(charts_in_scope[0])]].values[obs_n]) 

timestamp=1603824129
feature vector for 1th training observation:
[ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.         -0.00062657  0.          0.          0.0837521   0.
  0.          0.08312553  0.          0.          0.          0.
  0.         -0.0018797  -0.16792613 -0.0012626   0.00125627  0.08396307
 -0.0835422  -0.0856318   0.1679261   0.08270047 -0.0816583   0.1679261 ]


If you look close enough at the above two cells you will see the same values be shifted for each lag.

Each matrix of numbers above is the representation we give to our model of each timestep. This is how the model views each chart - a matrix (or "feature vector" if you want to sound fancy) or floating point numbers encoding some differenced and smoothed information about the last `lags_n` observations for each dimension in the specific chart we are modelling. 

## Get pediction data

Now that we have our trained models for each chart we can use them in looking at incoming obsevarions and 'ask' the trained models how 'unusual' it thinks they are. 

In [177]:
# define a empty dataframe we can store enough recent data into to generate our feature vector for recent data on
df_recent = pd.DataFrame()
times = []
# simulate n_prediction_steps of getting latest data, making feature vecotr and getting predicitons
for prediction_step in range(n_prediction_steps):
    time.sleep(1)
    df_latest = get_allmetrics(host=host, charts=charts_in_scope, wide=True)[df_train.columns]
    df_latest['time_idx'] = int(time.time())
    df_latest = df_latest.set_index('time_idx')
    # just keep enough recent data to generate each feature vector
    df_recent = df_recent.append(df_latest).tail((lags_n + smooth_n + diffs_n) * 2)
    df_predict_processed = make_features(df_recent, lags_n, diffs_n, smooth_n)
print(f'we now have {df_predict_processed.shape[0]} recent preprocessed feature vectors to predict on.')

we now have 10 recent preprocessed feature vectors to predict on.


In [178]:
print(df_predict_processed.shape)
df_predict_processed.head()

(10, 72)


,system.cpu|guest_lag0,system.cpu|guest_lag1,system.cpu|guest_lag2,system.cpu|guest_lag3,system.cpu|guest_lag4,system.cpu|guest_lag5,system.cpu|guest_nice_lag0,system.cpu|guest_nice_lag1,system.cpu|guest_nice_lag2,system.cpu|guest_nice_lag3,...,system.load|load1_lag2,system.load|load1_lag3,system.load|load1_lag4,system.load|load1_lag5,system.load|load5_lag0,system.load|load5_lag1,system.load|load5_lag2,system.load|load5_lag3,system.load|load5_lag4,system.load|load5_lag5
time_idx,,,,,,,,,,,,,,,,,,,,,
1603838531,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1603838532,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1603838533,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1603838535,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1603838536,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-0.003333,0.0,0.0,0.0,0.0,0.0


## Get predictions

In [179]:
# for each recent feature vector, get a prediction
for time_idx, row in df_predict_processed.iterrows():
    print(f'\npredictions for time {time_idx}\n')
    df_tmp = row.to_frame().transpose()
    for model in models:
        X_predict = df_tmp[df_tmp.columns[df_tmp.columns.str.startswith(model)]].values
        anomaly_probability = round(models[model].predict_proba(X_predict)[-1][1],4)
        anomaly_flag = models[model].predict(X_predict)[-1]
        print(f'model={model}, anomaly_probability={anomaly_probability}, anomaly_flag={anomaly_flag}')


predictions for time 1603838531

model=system.cpu, anomaly_probability=0.0591, anomaly_flag=0
model=system.load, anomaly_probability=0.0, anomaly_flag=0

predictions for time 1603838532

model=system.cpu, anomaly_probability=0.0588, anomaly_flag=0
model=system.load, anomaly_probability=0.0, anomaly_flag=0

predictions for time 1603838533

model=system.cpu, anomaly_probability=0.0683, anomaly_flag=0
model=system.load, anomaly_probability=0.0, anomaly_flag=0

predictions for time 1603838535

model=system.cpu, anomaly_probability=0.0699, anomaly_flag=0
model=system.load, anomaly_probability=0.0, anomaly_flag=0

predictions for time 1603838536

model=system.cpu, anomaly_probability=0.0717, anomaly_flag=0
model=system.load, anomaly_probability=0.0173, anomaly_flag=0

predictions for time 1603838537

model=system.cpu, anomaly_probability=0.093, anomaly_flag=0
model=system.load, anomaly_probability=0.0309, anomaly_flag=0

predictions for time 1603838538

model=system.cpu, anomaly_probability